In [1]:
import numpy as np
import altair as alt
import pandas as pd
import streamlit as st

alt.renderers.enable('notebook')

ValueError: 
To use the 'notebook' renderer, you must install the vega package
and the associated Jupyter extension.
See https://altair-viz.github.io/getting_started/installation.html
for more information.


In [2]:
def force(xx):
    global t, f1, f2
    xx = xx/t
    return f1*xx*((f2 - xx) * (f2 - xx/2) + 1)
    # return (f2 - xx) * (f2 - xx/2) + 1


def stiffness(xx):
    global t, f1, f2
    xx = xx/t
    st1 = f1*((f2 - xx) * (f2 - xx / 2) + 1)
    st2 = f1*xx*(-(f2 - xx/2) - (f2 - xx)/2)
    return (st1 + st2)/t


def stress(xx):
    global t, s1, f2, beta, gamma
    xx = xx/t
    return s1*xx*(beta*(f2-xx/2) + gamma)

In [3]:
def main():
    global E, t, nu, D, d, H
    global beta, gamma, f1, f2, s1
    # Constants ===============================================================
    h = H - t       # Height of truncated cone [m]
    delta = D/d     # Diameter ratio [-]

    alpha = ((delta - 1)/delta)**2 / \
            ((delta + 1)/(delta - 1) -
             2/np.log(delta)) / np.pi   # Auxiliary variable [-]

    beta = (6*((delta-1)/np.log(delta) - 1)
            / np.log(delta))/np.pi      # Auxiliary variable [-]

    gamma = (3*(delta-1) /
             np.pi)/np.log(delta)       # Auxiliary variable [-]

    # Force terms --------------------------------------------------
    f1 = 4*E*t**4/((1-nu**2)*alpha*D**2)
    f2 = h/t

    # Stress terms -------------------------------------------------
    s1 = 4*E*t**2/((1-nu**2)*alpha*D**2)

    # Calculations ============================================================
    x = np.linspace(0, h, 200)      # Deflection vector [m]
    F = force(x)
    k = stiffness(x)
    sigma = stress(x)
    df = pd.DataFrame({'Deflection [m]': x,
                       'Force [N]': F,
                       'Stiffness [N/m]': k,
                       'Stress [Pa]': sigma
                       })

    # Plots ===================================================================
    fig1 = alt.Chart(df).mark_line().encode(x='Deflection [m]:Q', y='Force [N]:Q').interactive()
    fig2 = alt.Chart(df).mark_line().encode(x='Deflection [m]:Q', y='Stiffness [N/m]:Q').interactive()
    fig3 = alt.Chart(df).mark_line().encode(x='Deflection [m]:Q', y='Stress [Pa]:Q').interactive()

    return fig1, fig2, fig3

In [4]:
E = 210e9       # Young's modulus [Pa]
t = .9e-3       # Washer thickness [m]
nu = 0.29       # Poisson's ratio [-]
D = 16e-3       # Outside diameter [m]
d = 8.2e-3      # Inside diameter [m]
H = 1.25e-3     # Washer height [m]

main()

(alt.Chart(...), alt.Chart(...), alt.Chart(...))